In [24]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import os
import numpy as np
from Training import GetDatasets
from Nets  import GetNeuralNetworkModel
from Stats_original import WeightQuantization
from Simulation import get_all_outputs

tf.random.set_seed(1234)
np.random.seed(1234)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
## Esta funcion convierte un valor de flotante a su equivalente binario de signo magnitud, asumiendo 11 bits
## para parte fraccionaria (los usados en AlexNet 
def convert(value):
    shift   = 2**(16-1)
    factor  = 2**11
    new_val = value*factor
    new_val = -new_val + shift if new_val < 0 else new_val
    new_val = int(new_val)
    return format(new_val, '#018b')

In [26]:
## Direccion del archivo con los pesos guardados
cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'AlexNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')

## Obtencion del dataset
train_batchSize = test_batchSize = 1
_,_,test_set = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, train_batchSize, test_batchSize)

# Se crea la red Alexnet con tres errores (en las direcciones 0,2 y 3)
posiciones      = [4593, 4661, 4685, 4737, 4789]
tipo_de_errores = ['x0xxxxxxx1xxxxxx', 'x1xxxxxxx0xxxxxx', 'x1xxxxxxx1xxxxxx', 'x1xxxxxxx0xxxxxx', 'x1xxxxxxx1xxxxxx']
QAlexNet  = GetNeuralNetworkModel('AlexNet',(227,227,3),8, posiciones, tipo_de_errores, 
                                  quantization = True, aging_active=True, batch_size = 1,
                                  word_size = 16, frac_size = 11)
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
metrics = ['accuracy']
QAlexNet.compile(optimizer=optimizer, loss=loss, metrics=metrics)
QAlexNet.load_weights(wgt_dir).expect_partial()

# Cuantizacion de los pesos
WeightQuantization(model = QAlexNet, frac_bits = 11, int_bits = 4)

AlexNet
index_list [[0, 20, 53, 0],
 [0, 20, 121, 0],
 [0, 20, 145, 0],
 [0, 20, 197, 0],
 [0, 21, 22, 0]]
mod_list [[49151, 64], [65471, 16384], [65535, 16448], [65471, 16384], [65535, 16448]]
dentro de active
dentro de apply round
tensor entero valores afectados  Tensor("lambda_79/SelectV2:0", shape=(5,), dtype=int32)
tensor dentro de RoundValues Tensor("lambda_79/SelectV2:0", shape=(5,), dtype=int32)
tensor valores_afectados Tensor("lambda_79/NotEqual:0", shape=(5,), dtype=bool)
valores malos Tensor("lambda_79/GatherNd_1:0", shape=(None,), dtype=int32)
valores buenos tf.print Tensor("lambda_79/GatherNd_2:0", shape=(None,), dtype=int32)
newValues DESPUES de colocar 0 Tensor("lambda_79/SelectV2_17:0", shape=(None,), dtype=int32)
tensor_act- antes de actualizar Tensor("lambda_79/SelectV2:0", shape=(5,), dtype=int32)
tensor_act que envio para inyectar errores nuevamente Tensor("lambda_79/TensorScatterUpdate:0", shape=(5,), dtype=int32)
new_Values que llega de redondeo Tensor("lambda_79/

mod_list [[49151, 64], [65471, 16384], [65535, 16448], [65471, 16384], [65535, 16448]]
dentro de active
dentro de apply round
tensor entero valores afectados  Tensor("lambda_96/SelectV2:0", shape=(5,), dtype=int32)
tensor dentro de RoundValues Tensor("lambda_96/SelectV2:0", shape=(5,), dtype=int32)
tensor valores_afectados Tensor("lambda_96/NotEqual:0", shape=(5,), dtype=bool)
valores malos Tensor("lambda_96/GatherNd_1:0", shape=(None,), dtype=int32)
valores buenos tf.print Tensor("lambda_96/GatherNd_2:0", shape=(None,), dtype=int32)
newValues DESPUES de colocar 0 Tensor("lambda_96/SelectV2_17:0", shape=(None,), dtype=int32)
tensor_act- antes de actualizar Tensor("lambda_96/SelectV2:0", shape=(5,), dtype=int32)
tensor_act que envio para inyectar errores nuevamente Tensor("lambda_96/TensorScatterUpdate:0", shape=(5,), dtype=int32)
new_Values que llega de redondeo Tensor("lambda_96/TensorScatterUpdate:0", shape=(5,), dtype=int32)
tensor luego de aplicar la inyección de errores última Ten

In [27]:
## veamos las primeras capas de alexnet
for i in range(5):
    print(QAlexNet.layers[i])

In [5]:
## Las primeras 5 capas son: Input->Quantizacion->Error+Correccion->Convolucion->Quantizacion,
## por ende si comparamos las salidas de la capa de quantizacion con las de Error+Correccion podremos comprobar si se aplico
## la aproximacion.

In [28]:
primera_imagen_dataset = next(iter(test_set))[0]
salidas_por_capa     = get_all_outputs(QAlexNet,primera_imagen_dataset)
salidas_quantizacion = salidas_por_capa[1][0]  ## segunda capa, primer elemento del batch
salidas_error_y_correccion = salidas_por_capa[2][0]  ## tercera capa, primer elemento del batch

dentro de active
dentro de apply round
tensor entero valores afectados  tf.Tensor([1960 1960 1944 1944 1944], shape=(5,), dtype=int32)
tensor dentro de RoundValues tf.Tensor([1960 1960 1944 1944 1944], shape=(5,), dtype=int32)
tensor_with_error [2024 18344 18392 18328 18392]
tensor valores_afectados tf.Tensor([ True  True  True  True  True], shape=(5,), dtype=bool)
valores malos tf.Tensor([ 2024 18344 18392 18328 18392], shape=(5,), dtype=int32)
valores buenos tf.print tf.Tensor([1960 1960 1944 1944 1944], shape=(5,), dtype=int32)
mask del for [64 16384 16384 16384 16384]
newValues ANTES  [1920 0 0 0 0]
newValues DESPUES de colocar 0 tf.Tensor([1920    0    0    0    0], shape=(5,), dtype=int32)
tensor_act- antes de actualizar tf.Tensor([1960 1960 1944 1944 1944], shape=(5,), dtype=int32)
tensor_act que envio para inyectar errores nuevamente tf.Tensor([1920    0    0    0    0], shape=(5,), dtype=int32)
new_Values que llega de redondeo tf.Tensor([1920    0    0    0    0], shape=(5,), 

tensor luego de aplicar la inyección de errores última tf.Tensor([-0.46875 -8.      -8.03125 -8.      -8.03125], shape=(5,), dtype=float32)
affectedValues [-0.448730469 -0.448730469 -0.448730469 -1.57470703 -1.57470703]
newValues para actualizar el tensor final [-0.46875 -8 -8.03125 -8 -8.03125]
velores del tensor final en index_list [-0.46875 -8 -8.03125 -8 -8.03125]
dentro de active
dentro de apply round
tensor entero valores afectados  tf.Tensor([2530 3560 3126 2028  913], shape=(5,), dtype=int32)
tensor dentro de RoundValues tf.Tensor([2530 3560 3126 2028  913], shape=(5,), dtype=int32)
tensor_with_error [2530 19880 19574 18348 17361]
tensor valores_afectados tf.Tensor([False  True  True  True  True], shape=(5,), dtype=bool)
valores malos tf.Tensor([19880 19574 18348 17361], shape=(4,), dtype=int32)
valores buenos tf.print tf.Tensor([3560 3126 2028  913], shape=(4,), dtype=int32)
mask del for [16384 16384 16384 16384]
newValues ANTES  [0 0 0 0]
newValues DESPUES de colocar 0 tf.Ten

In [7]:
## Para ver que indices de la capa son afectados por el error puedes hacer un tf.print(index_list) en la funcion Aging()
## definida en Nets.py, esto se printeara en runtime de tensorflow (osea al hacer una inferencia,
## como la que hace get_all_outputs)

## En este caso las direcciones 0,2,3 se traducen en los indices [0][0][0], [0][2][0], [0][3][0]

In [29]:
index_list [[0, 28, 28, 1], [0, 29, 41, 1], [0, 30, 10, 1], [0, 31, 7, 1], [0, 32, 4, 1]]

NameError: name 'index_list' is not defined

In [22]:
Valores_antes   = [salidas_quantizacion[0][0][0],salidas_quantizacion[0][2][0],salidas_quantizacion[0][3][0]]
Valores_despues = [salidas_error_y_correccion[0][0][0],salidas_error_y_correccion[0][2][0],salidas_error_y_correccion[0][3][0]]

for i in range(3):
    print('Error: ',tipo_de_errores[i])
    print('Valor antes: ',Valores_antes[i],'En binario: ',convert(Valores_antes[i]))
    print('Valor con error + aproximacion: ',Valores_despues[i],'En binario: ',convert(Valores_despues[i]))
    print('-------------------------')

Error:  x0xxxxxxx1xxxxxx
Valor antes:  0.953125 En binario:  0b0000011110100000
Valor con error + aproximacion:  0.953125 En binario:  0b0000011110100000
-------------------------
Error:  x1xxxxxxx0xxxxxx
Valor antes:  0.95214844 En binario:  0b0000011110011110
Valor con error + aproximacion:  0.95214844 En binario:  0b0000011110011110
-------------------------
Error:  x1xxxxxxx1xxxxxx
Valor antes:  0.94970703 En binario:  0b0000011110011001
Valor con error + aproximacion:  0.94970703 En binario:  0b0000011110011001
-------------------------


In [ ]:
0b0000011110100000
0b0000100000000000